In [2]:
import pandas as pd
import requests

In [5]:
print("Get artist informations")

# endpoint https://api.spotify.com/v1/artists/{id}

params_artist = {"id": "7jy3rLJdDQY21OgRLCZ9sD"}

url_artist = "https://api.spotify.com/v1/artists/"

#Appel API
try:
    response_artist = requests.get(url_artist, params_artist)

    #API artiste OK
    if response_artist.status_code == 200:
        print(response_artist)

        """if response_geo.json():
            data_geo = response_geo.json()[0]
            
            city["lat"] = float(data_geo["lat"])
            city["lon"] = float(data_geo["lon"])

            params_weather["lat"] = data_geo["lat"]
            params_weather["lon"] = data_geo["lon"]"""

except requests.exceptions.RequestException as error:
        print("Exception Artist Error:", error)     

Get artist informations
